In [14]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os

In [18]:
# 1. Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Class labels
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# 3. Load model (NO conv1 change)
model = models.resnet50()
model.fc = torch.nn.Linear(model.fc.in_features, 4)  # 4-class tumor classification

# Load weights
model.load_state_dict(torch.load('resnet50_trained.pth', map_location=device))
model.to(device)
model.eval()

/tmp/ipykernel_7237/3501687508.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('resnet50_trained.pth', map_location=device))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

def predict(image_path):
    image = Image.open(image_path)

    # Convert to RGB (needed since model expects 3 channels)
    if image.mode != 'RGB':
        image = image.convert('RGB')

    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.nn.functional.softmax(output[0], dim=0)
        pred = torch.argmax(probs).item()

    print(f"{os.path.basename(image_path)} → {class_names[pred]} ({probs[pred].item():.4f})")

In [22]:
predict("./dataset/AbsolutelyNewImages/new_glioma.jpeg")

new_glioma.jpeg → pituitary (0.9791)


In [23]:
predict("./dataset/AbsolutelyNewImages/new_meningioma.jpg")

new_meningioma.jpg → notumor (0.7547)


In [24]:
predict("./dataset/AbsolutelyNewImages/no_tumor.jpg")

no_tumor.jpg → notumor (0.8214)


In [25]:
predict("./dataset/AbsolutelyNewImages/pituitary.jpg")

pituitary.jpg → pituitary (0.9923)
